# Todo

# 1. print annotations
# 2. print language annotations
# 3. add ability to pick a subset containing specific languages (list of languages as parameter maybe?)



In [44]:


# dataset for sound
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import os
import json
import torchvision
import datetime
from subprocess import Popen, PIPE
import re

class soccernet_dataset_sound(Dataset):
    """Soccernet Dataset"""
    
    def __init__(self,npy_file,
                 root_dir):
        self.npy_file = np.load(npy_file)
        self.samples = list()
        self.root_dir = root_dir
        for e in self.npy_file:
            path, annotations = self.get_annotations(e)
            language = self.get_lang_label(e)

            for annotation in annotations:
                # Check that annotations hold correct labels
                        if ("card" in annotation["label"]) or ("subs" in annotation["label"]) or ("soccer" in annotation["label"]):
                            self.samples.append([path,annotation,"soccernet",language])


    def __len__(self):
        return len(self.samples)

    def __getitem__(self,idx):
        """Returns a sample containing video path, clip and label"""
        if torch.is_tensor(idx):
            idx.tolist()
        
        # get annotations
        time_half = int(self.samples[idx][1]["gameTime"][0])
        time_minute = int(self.samples[idx][1]["gameTime"][-5:-3])
        time_second = int(self.samples[idx][1]["gameTime"][-2:])
        annotation = self.samples[idx][1]
        

        # Get label
        if ("card" in annotation["label"]): label = 0
        elif ("subs" in annotation["label"]): label = 1
        elif ("soccer" in annotation["label"]): label = 2
        elif ("background" in annotation["label"]): label = 3
        else: 
            print("Warning, label not compatible with set")
            return
            
        vidpath = str(self.samples[idx][0])

        sample = {'vidpath': vidpath,'annotation':annotation,'label':label,'lang':self.samples[idx][3],'idx':idx}
        
        return sample
            
    def get_annotations(self,path):
        """ Reads json files and returns """
        with open(self.root_dir+path+"/Labels.json") as jsonfile:
            json_label = json.load(jsonfile)
        
        labels = [e for e in json_label['annotations']]
        
        return path,labels
    

    def get_lang_label(self,path):
        """ Reads json files and returns """
        with open(self.root_dir+path+"/lang_label_test.json") as jsonfile:
            json_label = json.load(jsonfile)
        
        return json_label


    def describe(self):
        card = 0
        subs = 0
        goal = 0
        background = 0

        for sample in self.samples:
            annotation = sample[1]
        # Get label
            if ("card" in annotation["label"]): card += 1
            elif ("subs" in annotation["label"]): subs +=1
            elif ("soccer" in annotation["label"]): goal += 1
            elif ("background" in annotation["label"]): background += 1

        print("Description of dataset\n\n")
        print("\n ********* Classes *********")
        print("\n card = 0\n subs = 1\n goals = 2\n background = 3")

        print("\n ********* Distribution and count *********")
        print(f"\n N card: {card} \n N subs: {subs} \n N goal: {goal} \n N background: {background} \n \n Total : {card+subs+goal+background}")
        
        print("\n\n ********* Configuration *********")
        print(f"\n npy_file: {self.npy_file} \n tshift: {self.tshift} \n root_dir: {self.root_dir} \n transform: {self.transform} \n frame_center: {self.frame_center} \n nframes: {self.nframes} \n stride_frames: {self.stride_frames} \n background: {self.background}")
        print("\n\n ********* End of description *********")

# Paths

In [45]:
root_dir = "/hdd/sound_sn/SoccerNet-code/data/"
train_file = "/hdd/SoccerNet-code/data/listgame_Train_300.npy"
valid_file = "/hdd/SoccerNet-code/data/listgame_Valid_100.npy"
test_file = "/hdd/SoccerNet-code/data/listgame_Test_100.npy"

# Test case

In [46]:
npy_file = train_file

In [47]:
sdata = soccernet_dataset_sound(root_dir=root_dir,npy_file=npy_file)

In [48]:
sdata

In [49]:
sdata[0]

{'vidpath': 'england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley',
 'annotation': {'gameTime': '1 - 13:10',
  'label': 'soccer-ball',
  'team': 'home'},
 'label': 2,
 'lang': {'lang': 'esp'},
 'idx': 0}

In [43]:
sdata[13]

{'vidpath': 'england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal',
 'annotation': {'gameTime': '1 - 45:07',
  'label': 'soccer-ball',
  'team': 'away'},
 'label': 2,
 'idx': 13}

# TODO : Create custom sampler for language, home away team etc

1. Need ability to sample based on language, team etc without creating horrible imbalances

# Load .wav

1. load .wav at specific point in time
    - Evaluate how much ram is used
    - visually look at wave format
    - Convert to Spectogram with window sizes of 10-15 ms
    
2. Train and test a simple model based on wave data or spectogram. ( .. classification task, with small temporal windows at first .. use dataloader as in clipdataset )

In [ ]:
import torchaudio